In [139]:
pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [140]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import Dataset, Reader, SVD
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise import accuracy

In [141]:
df = pd.read_csv("movies_dF.csv")
df.head()

,index,id,show_id,type,title,director,cast,country,date_added,release_year,rating_x,listed_in,description,duration_int,duration_type,userId,rating_y,movieId,avg_r
0,0,as1,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,113.0,min,543,5.0,as1,4.0
1,1,as2,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,110.0,min,321,3.0,as2,4.0
2,2,as3,s3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,74.0,min,582,5.0,as3,4.0
3,3,as4,s4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,documentary,"pink breaks the mold once again, bringing her ...",69.0,min,151,1.5,as4,4.5
4,4,as5,s5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,"drama, fantasy",teenage matt banting wants to work with a famo...,45.0,min,24,5.0,as5,3.0


In [142]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userId', 'id', 'avg_r']], reader)

In [143]:
#Separamos los datos para entrenamiento y test
trainset, testset = train_test_split(data, test_size=0.25)

In [144]:
#Instanciamos SVD
algo = SVD()

In [145]:
#Entrenamos
algo.fit(trainset)

In [146]:
#Predecimos con los datos de testeo
pred = algo.test(testset)

In [147]:
#Hacemos pruebas
userId = "150"
id = "as4"
avg_r = 4.0
algo.predict(userId, id, avg_r, verbose=True)

user: 150        item: as4        r_ui = 4.00   est = 3.60   {'was_impossible': False}


Prediction(uid='150', iid='as4', r_ui=4.0, est=3.600343884103925, details={'was_impossible': False})

In [148]:
#Funcion para recomendación
def get_movie_recommendation(userId, id):
    # Obtenemos la predicción de la calificación
    prediction = algo.predict(userId, str(id))

    # Si la predicción es mayor o igual a 4, se recomienda la película
    if prediction.est >= 4:
        return "Recomendada", prediction.est
    else:
        return "No recomendada", prediction.est

In [149]:
id_usuario = 432
id_pelicula = "ns3149"
recommendation = get_movie_recommendation(id_usuario, id_pelicula)
print(f"La mejor recomendacion es: {recommendation}")

La mejor recomendacion es: ('No recomendada', 3.265688807865891)


In [150]:
from surprise import accuracy
accuracy.rmse(pred)

RMSE: 1.0651


1.065124854162075

# Segunda Prueba

In [151]:
N_filas = 100000
reader = Reader(rating_scale=(0.5,5))
data = Dataset.load_from_df(df[['userId', 'id', 'avg_r']][:N_filas], reader)

In [152]:
trainset, testset = train_test_split(data, test_size=.25)

In [153]:
model = SVD()

In [154]:
model.fit(trainset)

In [155]:
predictions = model.test(testset)

In [156]:
#Funcion para recomendación
def get_movie_recommendation(userId, id):
    # Obtenemos la predicción de la calificación
    prediction = model.predict(userId, str(id))

    # Si la predicción es mayor o igual a 4, se recomienda la película
    if prediction.est >= 4:
        return "Recomendada", prediction.est
    else:
        return "No recomendada", prediction.est

In [157]:
#Hacemos un ejemplo
id_usuario = 432
id_pelicula = "ns3149"
recommendation = get_movie_recommendation(id_usuario, id_pelicula)
print(f"La mejor recomendacion es: {recommendation}")

La mejor recomendacion es: ('No recomendada', 3.250112295386699)


In [158]:
accuracy.rmse(predictions)

RMSE: 1.0508


1.0508005637045736

# Prueba de otra tecnica

In [159]:
from surprise import SVDpp

In [160]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df[['userId', 'id', 'avg_r']][:N_filas], reader)

In [161]:
#Separamos los datos en 2
train, test = train_test_split(data, test_size=0.25)

#Instanciamos el algoritmo y entrenamos
svd = SVDpp()
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Entrenar el modelo con todo el conjunto de datos
trainset = data.build_full_trainset()
model.fit(trainset)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0656  1.0414  1.0680  1.0696  1.0594  1.0608  0.0103  
MAE (testset)     0.8529  0.8303  0.8464  0.8608  0.8439  0.8469  0.0102  
Fit time          0.75    0.50    0.44    0.47    0.47    0.53    0.11    
Test time         0.05    0.02    0.02    0.02    0.02    0.03    0.01    


In [162]:
#Funcion para recomendación
def get_movie_recommendation(userId, id):
    # Obtenemos la predicción de la calificación
    prediction = model.predict(userId, id)
    rating = prediction.est

    # Si la predicción es mayor o igual a 4, se recomienda la película
    if rating >= 4:
        return "Recomendada", rating
    else:
        return "No recomendada", rating

In [163]:
user_id = 123
movie_id = 456

recommendation= get_movie_recommendation(user_id, movie_id)
print(f"La mejor recomendacion es: {recommendation}")

La mejor recomendacion es: ('No recomendada', 3.651488485587119)


# Subir modelo SVDpp a Gradio 

In [ ]:
pip install gradio

In [167]:
import gradio as gr
inputs = [
    gr.inputs.Number(label="ID de usuario"),
    gr.inputs.Textbox(label="ID de película"),
]
outputs = gr.outputs.Textbox(label="Predicción")
interface = gr.Interface(fn=get_movie_recommendation, inputs=inputs, outputs=outputs, title="Sistema de recomendación de películas")
interface.launch()

/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:59: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>